# MultiSensor Dataset Preparation 
- Experiment data: March 2024. Aluminum, Laser-Wire DED
- Aurthor: Chen Lequn

### Notebook 3a: Feature selection, ML modelling and ablation study (feature-based audio model)
- Full Dataset: All features (from previous notebook 2b)
- Ablated Dataset 1: Features after removing those with Pearson correlation > 0.97 (within audio modality).
- Ablated Dataset 2: Features after removing those with Pearson correlation > 0.9 (within audio modality).
- Ablated Dataset 3: Features from Ablated Dataset 1 with MI > 0.05.
- Ablated Dataset 4: Features from Ablated Dataset 1 with MI > 0.1.

In [1]:
from sklearnex import patch_sklearn, config_context
import dpctl
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
# import wandb

In [3]:
import os
import glob
import sys
import math
import numpy as np
import pandas as pd
from pandas import DataFrame
import scipy as sp
import itertools
from collections import defaultdict
from pprint import pprint
import pickle
from joblib import dump, load

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import gca
from pylab import *
import seaborn as sns
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
%matplotlib inline

# Scikit-learn general
from sklearn import preprocessing, metrics, svm, datasets, tree, neighbors
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate, KFold, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, scale
from sklearn.ensemble import HistGradientBoostingClassifier

# Scikit-learn models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, SVR
from sklearn.multiclass import OneVsRestClassifier

# Imbalanced learn
from imblearn.ensemble import BalancedRandomForestClassifier, RUSBoostClassifier

# XGBoost and LightGBM
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

# Utilities
from scipy.stats import uniform, randint
from scipy.interpolate import griddata
from sklearn.utils import shuffle, resample, class_weight
from utils import *


In [4]:
PROJECT_ROOT_DIR = "../"
IMAGE_PATH = os.path.join(PROJECT_ROOT_DIR, "result_images", 'defect classification', 'ML-baseline', 'ablation study')
model_path = os.path.join(PROJECT_ROOT_DIR, "trained_models", 'defect classification', 'ML-baseline', 'ablation study')
scaler_path = os.path.join(PROJECT_ROOT_DIR, "saved_scalers", 'defect classification',  'ML-baseline', 'ablation study')
os.makedirs(IMAGE_PATH, exist_ok=True)
os.makedirs(model_path, exist_ok=True)
os.makedirs(scaler_path, exist_ok=True)


# Change to desirable location of the raw dataset
Multimodal_dataset_PATH = "/home/chenlequn/pan1/Dataset/Laser-Wire-DED-ThermalAudio-Dataset"
Annotation_file_path = os.path.join(Multimodal_dataset_PATH, "Annotation")
Dataset_path = os.path.join(Multimodal_dataset_PATH, 'Dataset')
final_audio_dataset = os.path.join(Multimodal_dataset_PATH, 'Dataset', "audio")
final_image_dataset = os.path.join(Multimodal_dataset_PATH, 'Dataset', "thermal_images")
              

## function for automatically save the diagram/graph into the folder 
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGE_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 2.50

In [5]:
# # Initialize wandb
# wandb.init(project="LW-DED_audio-classification")

## Step 1: Dataset preparation

In [6]:
# Read the data back from the HDF5 file into a new DataFrame
df_audio_dataset = pd.read_hdf(os.path.join(Dataset_path, 'df_audio_dataset_with_annotations(raw_audio).h5'), key='df')

df_audio_dataset

,audio_name,image_name,experiment_number,label_1,label_2,label_3,rms_energy,amplitude_envelope_mean,amplitude_envelope_std,zero_crossing_rate,...,mfcc_8_mean,mfcc_8_std,mfcc_9_mean,mfcc_9_std,mfcc_10_mean,mfcc_10_std,mfcc_11_mean,mfcc_11_std,mfcc_12_mean,mfcc_12_std
0,Exp_17_1_10.wav,Exp_17_1_10.jpg,17_1,LOF,NaN,NaN,0.031746,0.038982,0.014304,0.064399,...,-1.955670,7.565694,6.784209,13.263989,-4.241771,7.623967,10.760045,6.365362,-4.255302,6.843529
1,Exp_17_1_11.wav,Exp_17_1_11.jpg,17_1,LOF,NaN,NaN,0.046936,0.053492,0.018727,0.033787,...,7.764361,6.547935,10.695354,7.944485,0.243526,8.428101,10.609675,6.292929,-4.174980,7.595537
2,Exp_17_1_12.wav,Exp_17_1_12.jpg,17_1,LOF,NaN,NaN,0.024222,0.033361,0.013914,0.043991,...,0.100900,4.152504,1.597273,6.513865,-0.382829,5.145632,7.629467,9.252165,-2.734367,4.957322
3,Exp_17_1_2.wav,Exp_17_1_2.jpg,17_1,LOF,NaN,NaN,0.034430,0.041273,0.012582,0.071202,...,-7.307548,8.177770,14.585797,9.160913,-8.542147,8.239860,11.234342,7.819167,-10.741705,5.996997
4,Exp_17_1_3.wav,Exp_17_1_3.jpg,17_1,LOF,NaN,NaN,0.028317,0.034911,0.008767,0.108163,...,-7.398950,10.955479,11.769259,11.742193,-7.276566,8.582523,11.893761,7.696877,-10.677185,7.131041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6175,Exp_April_16_test_4_4.wav,Exp_April_16_test_4_4.jpg,April_16_test_4,Laser-off,Shielding Gas-off,NaN,0.012902,0.015071,0.005879,0.099093,...,2.168951,6.490287,1.928032,8.401035,-1.724736,10.288368,-0.060851,8.381465,-12.953316,5.754806
6176,Exp_April_16_test_4_1.wav,Exp_April_16_test_4_1.jpg,April_16_test_4,Laser-off,Shielding Gas-off,NaN,0.013222,0.015257,0.005407,0.084127,...,0.476796,9.227147,0.945553,7.366714,-1.322515,7.652929,-0.733449,5.396196,-14.786466,7.339162
6177,Exp_April_16_test_4_2.wav,Exp_April_16_test_4_2.jpg,April_16_test_4,Laser-off,Shielding Gas-off,NaN,0.012246,0.014805,0.005100,0.093878,...,-0.026498,7.503857,2.117017,7.400688,-2.097013,8.523264,0.141502,6.387466,-10.734591,8.097878
6178,Exp_April_16_test_4_3.wav,Exp_April_16_test_4_3.jpg,April_16_test_4,Laser-off,Shielding Gas-off,NaN,0.013367,0.016546,0.005384,0.087302,...,1.089101,6.276635,0.685349,5.679959,-5.039695,5.392179,-2.741401,4.763264,-13.847133,7.911690


In [7]:
# Extract labels
y = df_audio_dataset['label_1'].to_list()
y_array = np.array(y)
le = LabelEncoder()
le.fit(y)
Y_encoded = le.transform(y)

# Non-feature columns to be dropped
non_feature_columns = ['audio_name', 'image_name', 'experiment_number',
                       'label_1', 'label_2', 'label_3']

# Extract features for the full dataset
X_all = df_audio_dataset.drop(columns=non_feature_columns).to_numpy()

X_all.shape

(6180, 83)

In [8]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes = np.unique(y),
                                                 y = y)

class_weights_encoded = class_weight.compute_class_weight('balanced',
                                                 classes = np.unique(Y_encoded),
                                                 y = Y_encoded)


class_names = (le.classes_).tolist()
print(f"class weights: {class_weights}")
print(f"class weights encoded: {class_weights_encoded}")
print(f"class        : {class_names}")

class weights: [1.6284585  3.41436464 5.44493392 0.34515498 0.98800959]
class weights encoded: [1.6284585  3.41436464 5.44493392 0.34515498 0.98800959]
class        : ['Balling', 'Degradation', 'LOF', 'Laser-off', 'Non-defective']


In [9]:
# Given features based on previous analysis results
features_dropped_090 = ['spectral_centroid_mean', 'spectral_centroid_std',
       'spectral_complexity_std', 'spectral_contrast_0_mean',
       'spectral_contrast_0_std', 'spectral_contrast_1_mean',
       'spectral_contrast_1_std', 'spectral_contrast_2_mean',
       'spectral_contrast_2_std', 'spectral_contrast_3_mean',
       'spectral_contrast_3_std', 'spectral_contrast_4_mean',
       'spectral_contrast_4_std', 'spectral_contrast_5_mean',
       'spectral_valley_0_std', 'spectral_valley_1_std',
       'spectral_valley_2_mean', 'spectral_valley_2_std',
       'spectral_valley_3_std', 'spectral_valley_4_std',
       'spectral_valley_5_mean', 'spectral_energy_band_ratio_std',
       'spectral_flatness_mean', 'spectral_flatness_std',
       'spectral_rolloff_std', 'spectral_strong_peak_mean',
       'spectral_strong_peak_std', 'spectral_variance_mean',
       'spectral_variance_std', 'spectral_skewness_std',
       'spectral_kurtosis_std', 'spectral_crest_factor_mean',
       'spectral_crest_factor_std', 'mfcc_0_std', 'mfcc_1_mean', 'mfcc_1_std',
       'mfcc_2_mean', 'mfcc_2_std', 'mfcc_3_mean', 'mfcc_3_std', 'mfcc_4_mean',
       'mfcc_4_std', 'mfcc_5_mean', 'mfcc_5_std', 'mfcc_6_mean', 'mfcc_6_std',
       'mfcc_7_mean', 'mfcc_7_std', 'mfcc_8_mean', 'mfcc_8_std', 'mfcc_9_mean',
       'mfcc_9_std', 'mfcc_10_mean', 'mfcc_10_std', 'mfcc_11_mean',
       'mfcc_11_std', 'mfcc_12_mean', 'mfcc_12_std']

features_dropped_097 = ['amplitude_envelope_std', 'zero_crossing_rate',
       'loudness_vickers', 'spectral_centroid_mean', 'spectral_centroid_std',
       'spectral_complexity_mean', 'spectral_complexity_std',
       'spectral_contrast_0_mean', 'spectral_contrast_0_std',
       'spectral_contrast_1_mean', 'spectral_contrast_1_std',
       'spectral_contrast_2_mean', 'spectral_contrast_2_std',
       'spectral_contrast_3_mean', 'spectral_contrast_3_std',
       'spectral_contrast_4_mean', 'spectral_contrast_4_std',
       'spectral_contrast_5_mean', 'spectral_contrast_5_std',
       'spectral_valley_0_mean', 'spectral_valley_0_std',
       'spectral_valley_1_mean', 'spectral_valley_1_std',
       'spectral_valley_2_mean', 'spectral_valley_2_std',
       'spectral_valley_3_mean', 'spectral_valley_3_std',
       'spectral_valley_4_mean', 'spectral_valley_4_std',
       'spectral_valley_5_mean', 'spectral_valley_5_std',
       'spectral_decrease_mean', 'spectral_energy_band_ratio_mean',
       'spectral_energy_band_ratio_std', 'spectral_flatness_mean',
       'spectral_flatness_std', 'spectral_flux_std', 'spectral_rolloff_mean',
       'spectral_rolloff_std', 'spectral_strong_peak_mean',
       'spectral_strong_peak_std', 'spectral_variance_mean',
       'spectral_variance_std', 'spectral_skewness_std',
       'spectral_kurtosis_std', 'spectral_crest_factor_mean',
       'spectral_crest_factor_std', 'mfcc_0_mean', 'mfcc_0_std', 'mfcc_1_mean',
       'mfcc_1_std', 'mfcc_2_mean', 'mfcc_2_std', 'mfcc_3_mean', 'mfcc_3_std',
       'mfcc_4_mean', 'mfcc_4_std', 'mfcc_5_mean', 'mfcc_5_std', 'mfcc_6_mean',
       'mfcc_6_std', 'mfcc_7_mean', 'mfcc_7_std', 'mfcc_8_mean', 'mfcc_8_std',
       'mfcc_9_mean', 'mfcc_9_std', 'mfcc_10_mean', 'mfcc_10_std',
       'mfcc_11_mean', 'mfcc_11_std', 'mfcc_12_mean', 'mfcc_12_std']


features_dropped_090_mi_filtered_01 = ['spectral_centroid_mean', 'spectral_centroid_std',
       'spectral_complexity_std', 'spectral_contrast_3_mean',
       'spectral_contrast_5_mean', 'spectral_valley_2_mean',
       'spectral_valley_5_mean', 'spectral_flatness_mean',
       'spectral_flatness_std', 'spectral_variance_mean',
       'spectral_skewness_std', 'spectral_crest_factor_mean',
       'spectral_crest_factor_std', 'mfcc_1_mean', 'mfcc_2_mean',
       'mfcc_3_mean', 'mfcc_5_mean', 'mfcc_7_mean', 'mfcc_9_mean',
       'mfcc_11_mean']


features_dropped_090_mi_filtered_005 = ['spectral_centroid_mean', 'spectral_centroid_std',
       'spectral_complexity_std', 'spectral_contrast_3_mean',
       'spectral_contrast_4_mean', 'spectral_contrast_5_mean',
       'spectral_valley_2_mean', 'spectral_valley_5_mean',
       'spectral_energy_band_ratio_std', 'spectral_flatness_mean',
       'spectral_flatness_std', 'spectral_rolloff_std',
       'spectral_strong_peak_std', 'spectral_variance_mean',
       'spectral_skewness_std', 'spectral_kurtosis_std',
       'spectral_crest_factor_mean', 'spectral_crest_factor_std',
       'mfcc_1_mean', 'mfcc_2_mean', 'mfcc_3_mean', 'mfcc_5_mean',
       'mfcc_6_mean', 'mfcc_7_mean', 'mfcc_7_std', 'mfcc_8_mean',
       'mfcc_9_mean', 'mfcc_9_std', 'mfcc_11_mean', 'mfcc_12_mean']

# Create ablated datasets
X_dropped_097 = df_audio_dataset[features_dropped_097].to_numpy()
X_dropped_090 = df_audio_dataset[features_dropped_090].to_numpy()
X_dropped_090_mi_filtered_01 = df_audio_dataset[features_dropped_090_mi_filtered_01].to_numpy()
X_dropped_090_mi_filtered_005 = df_audio_dataset[features_dropped_090_mi_filtered_005].to_numpy()


# Get the shapes of these additional ablated datasets
X_dropped_097.shape, X_dropped_090.shape, X_dropped_090_mi_filtered_005.shape, X_dropped_090_mi_filtered_01.shape,

((6180, 73), (6180, 58), (6180, 30), (6180, 20))

- ALL Dataset (X_all):  `83 features`
- Ablated Dataset 1 (X_dropped_097):  `73 features`
- Ablated Dataset 2 (X_dropped_090):  `58 features`
- Ablated Dataset 3 (X_dropped_090_mi_filtered_005): `30 features`
- Ablated Dataset 4 (X_dropped_090_mi_filtered_01):  `20 features`

### Train val test split
- Train 80%, Val 10%, Test 10%

In [10]:
y_array = np.array(y)

# First split: Separate out the training set (80% of original)
train_indices, temp_indices = train_test_split(
    np.arange(len(df_audio_dataset)), test_size=0.2, random_state=0, stratify=y_array)

# Second split: Separate out the validation and test sets (each will be 10% of original)
val_indices, test_indices = train_test_split(
    temp_indices, test_size=0.5, random_state=0, stratify=y_array[temp_indices])

# Check the shape of the indices for train, val, and test splits
len(train_indices), len(val_indices), len(test_indices)

(4944, 618, 618)

In [11]:
%store train_indices
%store val_indices
%store test_indices

Stored 'train_indices' (ndarray)
Stored 'val_indices' (ndarray)
Stored 'test_indices' (ndarray)


## Step 2: Create a Pipeline and Iterate Over Datasets and Models

In [12]:
# Define hyperparameters for each model
hyperparameters = {
    'KNN': {'n_neighbors': 12, 'weights': 'distance'},
    'DT': {'ccp_alpha': 3.89e-05, 'max_depth': 21, 'min_samples_leaf': 1, 'min_samples_split': 13},
    'LR': {'C': 5.15, 'max_iter': 2528, 'penalty': 'l2', 'solver': 'lbfgs'},
    'RF': {'bootstrap': False, 'max_depth': 64, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 6, 'n_estimators': 307},
    'BalancedRF': {'bootstrap': False, 'max_depth': 23, 'max_features': 'sqrt', 'min_samples_leaf': 8, 'min_samples_split': 17, 'n_estimators': 237},
    'RUSBoost': {'learning_rate': 0.1, 'n_estimators': 150},
    'Ada': {'algorithm': 'SAMME', 'learning_rate': 0.57, 'n_estimators': 307},
    'GP': {'kernel': 1**2 * RBF(length_scale=1)},
    'NN': {'activation': 'relu', 'alpha': 0.077, 'early_stopping': True, 'hidden_layer_sizes': (229, 255, 168), 'learning_rate': 'adaptive', 'max_iter': 11805, 'solver': 'adam', 'validation_fraction': 0.1},
    'XGBoost': {'colsample_bytree': 0.62, 'gamma': 0.48, 'learning_rate': 0.35, 'max_depth': 44, 'min_child_weight': 8, 'n_estimators': 480, 'subsample': 0.59,  'gpu_id': 0},
    'LightGBM': {'colsample_bytree': 0.80, 'learning_rate': 0.15, 'max_depth': 35, 'min_child_weight': 1, 'n_estimators': 276, 'num_leaves': 120, 'reg_alpha': 1.94, 'reg_lambda': 1.70, 'subsample': 0.86, 'verbose': -1},
    'SVM': {'C': 524.86, 'degree': 9, 'gamma': 0.029, 'kernel': 'rbf', 'probability': True},
    'GB': {'learning_rate': 0.23, 'max_depth': 49, 'min_samples_leaf': 10, 'min_samples_split': 17, 'n_estimators': 271, 'n_iter_no_change': 10, 'subsample': 0.73, 'tol': 0.0001},
    'HistGB': {'learning_rate': 0.23, 'max_iter': 100, 'max_depth': 49, 'min_samples_leaf': 10, 'n_iter_no_change': 10, 'tol': 0.0001},
}

# List of datasets
datasets = {
    'All Features': X_all,
    'Ablated 1': X_dropped_097,
    'Ablated 2': X_dropped_090,
    'Ablated 3': X_dropped_090_mi_filtered_005,
    'Ablated 4': X_dropped_090_mi_filtered_01,
}


# List of models
models = {
    'KNN': KNeighborsClassifier,
    'DT': DecisionTreeClassifier,
    'LR': LogisticRegression,
    'RF': RandomForestClassifier,
    'BalancedRF': BalancedRandomForestClassifier,
    'RUSBoost': RUSBoostClassifier,
    'Ada': AdaBoostClassifier,
    # 'GP': GaussianProcessClassifier,
    'NN': MLPClassifier,
    'XGBoost': XGBClassifier,
    'LightGBM': LGBMClassifier,
    'SVM': SVC,
    # 'GB': GradientBoostingClassifier,
    'HistGB': HistGradientBoostingClassifier
}

## Step 3: Model Training

In [13]:
# Initialize KFold with the number of splits you desire
kf = KFold(n_splits=5)
# Create a dictionary to store final results
final_results = defaultdict(list)
# Create a dictionary to store temporary cross-validation results for each dataset and model
cv_results = defaultdict(lambda: defaultdict(list))


# Iterate over datasets and models
for dataset_name, X in datasets.items():
    print(f"Processing dataset: {dataset_name}")
    # Split data
    X_train = X[train_indices]
    X_val = X[val_indices]
    X_test = X[test_indices]
  
    y_train = Y_encoded[train_indices]
    y_val = Y_encoded[val_indices]
    y_test = Y_encoded[test_indices]

    # Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    with open(os.path.join(scaler_path, f'StandardScaler_{dataset_name}.pkl'), 'wb') as file:
        pickle.dump(scaler, file)

    for model_name, Model in models.items():
        print(f"\tTraining model: {model_name}")
        # with wandb.init(project="LW-DED_audio-classification", name=f"{model_name}_{dataset_name}", reinit=True) as run:
            # Log model and hyperparameters
            # wandb.config.update({"Model": model_name, "Hyperparameters": hyperparameters[model_name]})
    
        # Initialize model with the best hyperparameters
        model = Model(**hyperparameters[model_name])

        # If the model is SVM, use only 20% of the training data
        if model_name == 'SVM':
            sample_indices = np.random.choice(len(X_train), int(0.2 * len(X_train)), replace=False)
            X_train_sampled = X_train[sample_indices]
            y_train_sampled = y_train[sample_indices]
            model.fit(X_train_sampled, y_train_sampled)
        else:
            # Train the model
            model.fit(X_train, y_train)

        # Save the model
        dump(model, os.path.join(model_path, f'{model_name}_{dataset_name}.joblib'))


        #####------   cross-validation on the validation dataset--------------
        fold_metrics = defaultdict(list)

        for train_idx, test_idx in kf.split(X_val):
            X_test_train, X_test_val = X_val[train_idx], X_val[test_idx]
            y_test_train, y_test_val = y_val[train_idx], y_val[test_idx]
            
            # Make predictions
            y_test_pred = model.predict(X_test_val)
            
            # Evaluate the model
            acc = accuracy_score(y_test_val, y_test_pred)
            f1 = f1_score(y_test_val, y_test_pred, average='weighted')
            precision = precision_score(y_test_val, y_test_pred, average='weighted')
            recall = recall_score(y_test_val, y_test_pred, average='weighted')
            
            # Optional: ROC-AUC can only be calculated for binary classification or multilabel indicator format
            try:
                roc_auc = roc_auc_score(y_test_val, model.predict_proba(X_test_val), multi_class='ovr', average='weighted')
                
            except:
                roc_auc = 'N/A'
            
            # Store the metrics for this fold
            fold_metrics['Accuracy'].append(acc)
            fold_metrics['F1 Score'].append(f1)
            fold_metrics['Precision'].append(precision)
            fold_metrics['Recall'].append(recall)
            fold_metrics['ROC-AUC'].append(roc_auc if roc_auc != 'N/A' else np.nan)
        
        # Calculate mean and std for each metric and print them
        for metric, values in fold_metrics.items():
            mean_val = np.nanmean(values)  # Handle 'N/A' values
            std_val = np.nanstd(values)    # Handle 'N/A' values
            
            print(f"\t\t{metric}: Mean = {mean_val}, Std = {std_val}")
            # Log the mean and std values to wandb
            # wandb.log({f"{metric}_mean": mean_val, f"{metric}_std": std_val})
            
            cv_results[dataset_name][f"{model_name}_{metric}_mean"] = mean_val
            cv_results[dataset_name][f"{model_name}_{metric}_std"] = std_val
             
            #####------   cross-validation on the validation dataset--------------
            
# Convert the temporary results to final format
for dataset_name, metrics in cv_results.items():
    final_results['Dataset'].append(dataset_name)
    for metric_name, value in metrics.items():
        final_results[metric_name].append(value)

# Convert results to a DataFrame for easier viewing
final_results_df = pd.DataFrame(final_results)

Processing dataset: All Features
	Training model: KNN
		Accuracy: Mean = 0.9272095462890112, Std = 0.010058082015614327
		F1 Score: Mean = 0.9212034152065314, Std = 0.014344820909781637
		Precision: Mean = 0.926148872473242, Std = 0.02288499237970134
		Recall: Mean = 0.9272095462890112, Std = 0.010058082015614327
		ROC-AUC: Mean = 0.9894899313115326, Std = 0.0022046901353504657
	Training model: DT


/home/chenlequn/anaconda3/envs/working_primary/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


		Accuracy: Mean = 0.9126278520849723, Std = 0.010646669553943909
		F1 Score: Mean = 0.9140583704474128, Std = 0.009736032904553063
		Precision: Mean = 0.9208044864757972, Std = 0.009621261992551245
		Recall: Mean = 0.9126278520849723, Std = 0.010646669553943909
		ROC-AUC: Mean = 0.9603916736290377, Std = 0.007237132564175337
	Training model: LR
		Accuracy: Mean = 0.9304091266719119, Std = 0.008321312980090183
		F1 Score: Mean = 0.9256976175778915, Std = 0.013435257708990246
		Precision: Mean = 0.9280350788779484, Std = 0.021140124339097816
		Recall: Mean = 0.9304091266719119, Std = 0.008321312980090183
		ROC-AUC: Mean = 0.9905348914600172, Std = 0.0031431696463384818
	Training model: RF


/home/chenlequn/anaconda3/envs/working_primary/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chenlequn/anaconda3/envs/working_primary/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chenlequn/anaconda3/envs/working_primary/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

		Accuracy: Mean = 0.9352871754523997, Std = 0.007113866459614365
		F1 Score: Mean = 0.9290979767710092, Std = 0.012824844857011124
		Precision: Mean = 0.9331867135049816, Std = 0.02421617138195221
		Recall: Mean = 0.9352871754523997, Std = 0.007113866459614365
		ROC-AUC: Mean = 0.993972008163259, Std = 0.0023024278119924855
	Training model: BalancedRF
		Accuracy: Mean = 0.9093102543928666, Std = 0.027010794622551827
		F1 Score: Mean = 0.9155905562984984, Std = 0.024546559086813437
		Precision: Mean = 0.927544497211614, Std = 0.021556685157103548
		Recall: Mean = 0.9093102543928666, Std = 0.027010794622551827
		ROC-AUC: Mean = 0.9910966602941331, Std = 0.003908368979427452
	Training model: RUSBoost
		Accuracy: Mean = 0.8171518489378442, Std = 0.01318791888260022
		F1 Score: Mean = 0.8298862744955342, Std = 0.013864062988829919
		Precision: Mean = 0.8739835378306102, Std = 0.01792005699221271
		Recall: Mean = 0.8171518489378442, Std = 0.01318791888260022
		ROC-AUC: Mean = 0.937460947772

...

		Accuracy: Mean = 0.9400996590611068, Std = 0.011164776591806803
		F1 Score: Mean = 0.9367533162619681, Std = 0.015329501344032591
		Precision: Mean = 0.9423851409443589, Std = 0.023079598967548652
		Recall: Mean = 0.9400996590611068, Std = 0.011164776591806803
		ROC-AUC: Mean = 0.9922704898978557, Std = 0.0027968158310161134
Processing dataset: Ablated 4
	Training model: KNN
		Accuracy: Mean = 0.93688696564385, Std = 0.006130901015422586
		F1 Score: Mean = 0.9317712379102596, Std = 0.007861646265451399
		Precision: Mean = 0.9400873805559702, Std = 0.005973618108359092
		Recall: Mean = 0.93688696564385, Std = 0.006130901015422586
		ROC-AUC: Mean = 0.9875237699228988, Std = 0.0038711406595564953
	Training model: DT
		Accuracy: Mean = 0.8818777865198006, Std = 0.02077381950933902
		F1 Score: Mean = 0.8859471782294343, Std = 0.022582613054739274
		Precision: Mean = 0.8967345906672547, Std = 0.02706617324199423
		Recall: Mean = 0.8818777865198006, Std = 0.02077381950933902
		ROC-AUC: Mean

/home/chenlequn/anaconda3/envs/working_primary/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chenlequn/anaconda3/envs/working_primary/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/chenlequn/anaconda3/envs/working_primary/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to si

		Accuracy: Mean = 0.9368738526094939, Std = 0.014958886124232533
		F1 Score: Mean = 0.9336844700617546, Std = 0.018480612834113026
		Precision: Mean = 0.9380728768722809, Std = 0.0242689973201434
		Recall: Mean = 0.9368738526094939, Std = 0.014958886124232533
		ROC-AUC: Mean = 0.9937800170337366, Std = 0.002105079381816053
	Training model: BalancedRF
		Accuracy: Mean = 0.9109756097560975, Std = 0.013702236214362961
		F1 Score: Mean = 0.9144674232014802, Std = 0.013206156096191718
		Precision: Mean = 0.9234828506992224, Std = 0.012436993497934057
		Recall: Mean = 0.9109756097560975, Std = 0.013702236214362961
		ROC-AUC: Mean = 0.990936432015349, Std = 0.004073094629279369
	Training model: RUSBoost
		Accuracy: Mean = 0.8235772357723578, Std = 0.02407038101847037
		F1 Score: Mean = 0.830683616699092, Std = 0.018183035885375527
		Precision: Mean = 0.865211147150626, Std = 0.01585799647682165
		Recall: Mean = 0.8235772357723578, Std = 0.02407038101847037
		ROC-AUC: Mean = 0.961017767633856

/home/chenlequn/anaconda3/envs/working_primary/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chenlequn/anaconda3/envs/working_primary/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chenlequn/anaconda3/envs/working_primary/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

		Accuracy: Mean = 0.8883162863886703, Std = 0.009813036555574412
		F1 Score: Mean = 0.8792097407245038, Std = 0.010177109746595173
		Precision: Mean = 0.8808330558424267, Std = 0.017733398557429825
		Recall: Mean = 0.8883162863886703, Std = 0.009813036555574412
		ROC-AUC: Mean = 0.9774834887861765, Std = 0.0037125822517845336
	Training model: NN
		Accuracy: Mean = 0.9304091266719118, Std = 0.008346794015523604
		F1 Score: Mean = 0.9308588929917894, Std = 0.006034688180472397
		Precision: Mean = 0.9383758705479801, Std = 0.006482991186295234
		Recall: Mean = 0.9304091266719118, Std = 0.008346794015523604
		ROC-AUC: Mean = 0.9920373111895655, Std = 0.0025658360804525324
	Training model: XGBoost
		Accuracy: Mean = 0.9288486755835301, Std = 0.014602846036149648
		F1 Score: Mean = 0.9272965443336997, Std = 0.01566114018656364
		Precision: Mean = 0.9355027324675973, Std = 0.021641748413667554
		Recall: Mean = 0.9288486755835301, Std = 0.014602846036149648
		ROC-AUC: Mean = 0.992092260410899

In [14]:
%store datasets
%store le
%store Y_encoded

Stored 'datasets' (dict)
Stored 'le' (LabelEncoder)
Stored 'Y_encoded' (ndarray)
